In [1]:
from glob import glob
import os
import pandas as pd
from tqdm import tqdm
import json
from tokenizers import ByteLevelBPETokenizer
import random

In [2]:
#import json

# Specify the path to your JSON file
json_file_path = "data_dd_all_0418_quarter.json"

# Open the JSON file
with open(json_file_path, "r") as json_file:
    # Load the contents of the file
    data = json.load(json_file)


In [52]:
# Shuffle the data
random.shuffle(lines)

# Split the data into train, validation, and test sets
num_examples = len(lines)
num_train = int(num_examples * 0.7)  # 70% for training
num_val = int(num_examples * 0.2)  # 20% for validation
num_test = num_examples - num_train - num_val  # rest for test

train_data = lines[:num_train]
val_data = lines[num_train:num_train+num_val]
test_data = lines[num_train+num_val:]

# Save train, validation, and test sets to text files
with open("train_data/rain.txt", "w") as f:
    for example in train_data:
        f.write(example + "\n")

with open("train_data/val.txt", "w") as f:
    for example in val_data:
        f.write(example + "\n")

with open("train_data/test.txt", "w") as f:
    for example in test_data:
        f.write(example + "\n")

In [45]:
file_path = "train_data/test_0148.txt"

with open(file_path, "r") as file:
    test_data = file.read()

In [38]:
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2TokenizerFast
from transformers import GPT2Tokenizer
from datasets import load_dataset
import json
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import  DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
from datasets import load_from_disk

/home/nsml/.conda/envs/0418_test/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
# Wrap the custom tokenizer with the Hugging Face GPT2TokenizerFast
tokenizer = GPT2Tokenizer.from_pretrained("test_tokenizer")

In [41]:
tokenizer.add_special_tokens({"bos_token": "<s>", "eos_token": "</s>", "unk_token": "<unk>", "pad_token": "<pad>", "mask_token": "<mask>"})
#tokenizer.encode(data_dd_all_0418_quarter[0])

0

In [42]:
def tokenize(tokenizer, dataset):
    
    def encode_with_special_tokens(examples):
        bos_token = tokenizer.encode("<s>")
        eos_token = tokenizer.encode("</s>")
        encoded_sequences = [
            bos_token + tokenizer.encode(sequence, truncation=True, max_length=800) + eos_token
            for sequence in examples["text"]
        ]
        return {"input_ids": encoded_sequences}

    tokenized_dataset = dataset.map(
        encode_with_special_tokens,
        batched=True,
        num_proc=1,
        remove_columns=["text"],
    )

    return tokenized_dataset

In [53]:
# Loading the dataset
text_datasets = {
    "train": ['train_data/rain.txt'],
    "eval": ['train_data/val.txt'],
    "test": ['train_data/test.txt']
}

dataset = load_dataset("text", data_files=text_datasets, cache_dir="token_data")

# Tokenizing the dataset
tokenized_dataset = tokenize(tokenizer, dataset)

# Saving the tokenized dataset
tokenized_dataset.save_to_disk('token_data')


Using custom data configuration default-be4d965029f64e72


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 92.28it/s]


Dataset text downloaded and prepared to token_data/text/default-be4d965029f64e72/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


100%|██████████| 714/714 [03:55<00:00,  3.04ba/s]
